In [ ]:
!git clone https://github.com/openai/shap-e.git

In [1]:
%cd shap-e

/content/shap-e


In [ ]:
!pip install -e .

In [ ]:
!pip install langchain langchain-community
!pip install langchain-text-splitters
!pip install langchain-classic
!pip install -U langchain_ollama

In [ ]:
!pip install chromadb trimesh unstructured

In [ ]:
# =========================
# LangChain (API ACTUAL)
# =========================
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_classic.chains import create_retrieval_chain
from langchain_ollama import OllamaEmbeddings, ChatOllama

# =========================
# Core libs
# =========================
import torch
import trimesh

# =========================
# Shap-E
# =========================
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import (
    decode_latent_mesh,
    create_pan_cameras,
    decode_latent_images,
    gif_widget,
)


In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import torch
import trimesh



from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget, decode_latent_mesh

In [ ]:
# ===========================================
# cargar y dividir documentos 3D (texto)
# ===========================================
loader = DirectoryLoader("./3d_docs", glob="**/*.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200
)

chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


FileNotFoundError: Directory not found: './3d_docs'

In [ ]:
# ===========================================
# crear la base vectorial (ChromaDB)
# ===========================================
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings
)

vectorstore.add_documents(chunks)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:

# ===========================================
# cadena RAG: recuperar + generar prompt 3D
# ===========================================
ollama_url= "https://jonathan-enjoyed-rarely-bridges.trycloudflare.com"

llm = Ollama(
    model="mistral",
    base_url=ollama_url,
    )

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
)

query = "a large meat knife"
prompt_3d = qa_chain.run(query)

print("\n🔍 Prompt generado para Shap-E:\n", prompt_3d)


In [ ]:
# ===========================================
# generación 3D con Shap-E
# ===========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
try:
    x = torch.empty((1024, 1024, 256), device=device)
    print("✅ Tensor creado en GPU correctamente.")
except RuntimeError as e:
    print("❌ Error al asignar tensor:", e)

decoder = load_model("decoder", device=device)
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
batch_size = 4
guidance_scale = 15.0
latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[query] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=False,
    use_karras=True,
    karras_steps=256,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)
mesh = decode_latent_mesh(decoder, latents[0])
# Convertir de shap-e mesh → trimesh
# tri = trimesh.Trimesh(
#     vertices=mesh.verts.numpy(),
#     faces=mesh.faces.numpy(),
#     process=False
# )
# Convertir de shap-e mesh → trimesh (pasar primero a CPU)
tri = trimesh.Trimesh(
    vertices=mesh.verts.cpu().numpy(),
    faces=mesh.faces.cpu().numpy(),
    process=False
)

tri.export("knife.obj")

print("\n✅ Modelo 3D generado: router_cisco_1941.obj")